# Bundle Adjustment Example using PyPose and the BAL dataset

```
The dataset is from the following paper:  
Sameer Agarwal, Noah Snavely, Steven M. Seitz, and Richard Szeliski.  
Bundle adjustment in the large.  
In European Conference on Computer Vision (ECCV), 2010.  
```

Link to the dataset: https://grail.cs.washington.edu/projects/bal/

# Sparse-Jac

In [1]:
from bal_loader import build_pipeline

TARGET_DATASET = "ladybug"
TARGET_PROBLEM = "problem-49-7776-pre"

def filter_problem(x):
  return x['problem_name'] == TARGET_PROBLEM

dataset_pipeline = build_pipeline(dataset=TARGET_DATASET, cache_dir='bal_data').filter(filter_problem)
dataset_iterator = iter(dataset_pipeline)
dataset = next(dataset_iterator)

print(f'Fetched {TARGET_PROBLEM} from {TARGET_DATASET}')

import torch
import torch.nn as nn
import pypose as pp

# Load data
MAX_PIXELS = 31843 # the full dataset has 30k pixels, and we trim a fraction of it
DEVICE = 'cpu'

def trim_dataset(dataset, max_pixels):
  trimmed_dataset = dict()
  trimmed_dataset['points_2d'] = dataset['points_2d'][:max_pixels]
  trimmed_dataset['point_index_of_observations'] = dataset['point_index_of_observations'][:max_pixels]
  trimmed_dataset['camera_index_of_observations'] = dataset['camera_index_of_observations'][:max_pixels]
  # other fields are not changed
  trimmed_dataset['camera_extrinsics'] = dataset['camera_extrinsics']
  trimmed_dataset['camera_intrinsics'] = dataset['camera_intrinsics']
  trimmed_dataset['camera_distortions'] = dataset['camera_distortions']
  trimmed_dataset['points_3d'] = dataset['points_3d']

  for k in trimmed_dataset.keys():
    if not isinstance(trimmed_dataset[k], torch.Tensor):
      trimmed_dataset[k] = torch.from_numpy(trimmed_dataset[k])
    trimmed_dataset[k] = trimmed_dataset[k].to(DEVICE)
  return trimmed_dataset

def reprojerr(pose, points, pixels, intrinsics, distortions, camera_index, point_index):
  points = points[point_index, None] # [1000, 1, 3]
  pose = pose[camera_index] # [1000, 7]
  points = pose.unsqueeze(-2) @ points
  points = points.squeeze(-2)

  # perspective division
  points_proj = -points[:, :2] / points[:, -1:]

  # convert to pixel coordinates
  # intrinsics = intrinsics[camera_index]
  # distortions = distortions[camera_index]
  f = intrinsics[:, 0, 0]
  k1 = distortions[:, 0]
  k2 = distortions[:, 1]
  n = torch.sum(points_proj**2, dim=-1)
  r = 1.0 + k1 * n + k2 * n**2
  img_repj = f[:, None] * r[:, None] * points_proj

  # calculate the reprojection error
  loss = (img_repj - pixels).norm(dim=-1)

  return loss

trimmed_dataset = trim_dataset(dataset, max_pixels=MAX_PIXELS)

# errors = reprojerr(trimmed_dataset['camera_extrinsics'], # [49, 7], a LieTensor
#                    trimmed_dataset['points_3d'], # [7776, 3]
#                    trimmed_dataset['points_2d'], # [1000, 2] before trimmed [31843, 2]
#                    trimmed_dataset['camera_intrinsics'], # [49, 3, 3]
#                    trimmed_dataset['camera_distortions'],) # [49, 2
                  #  trimmed_dataset['point_index_of_observations'], # [1000] this 2-D point is produced by which 3-D point
                  #  trimmed_dataset['camera_index_of_observations']) # [1000] this 2-D point is produced by which camera


Streaming data for ladybug...
Fetched problem-49-7776-pre from ladybug


In [2]:
def reprojerr_vmap(pose, point, pixel, intrinsic, distortion):
  # reprojerr_vmap is not batched, it operates on a single 3D point and camera
  pose = pp.LieTensor(pose, ltype=pp.SE3_type) # pose will lose its ltype through vmap, temporary fix
  point = pose.unsqueeze(-2) @ point
  point = point.squeeze(-2)

  # perspective division
  point_proj = -point[:2] / point[-1:]

  # convert to pixel coordinates
  f = intrinsic[0, 0]
  k1 = distortion[0]
  k2 = distortion[1]
  n = torch.sum(point_proj**2, dim=-1)
  r = 1.0 + k1 * n + k2 * n**2
  img_repj = f * r * point_proj

  # calculate the reprojection error
  loss = (img_repj - pixel).norm(dim=-1)

  return loss

# jac_function_vmap = torch.vmap(pp.func.jacrev(reprojerr_vmap))
# jac_from_vmap = jac_function_vmap(trimmed_dataset['camera_extrinsics'][trimmed_dataset['camera_index_of_observations']],
#                       trimmed_dataset['points_3d'][trimmed_dataset['point_index_of_observations'], None],
#                       trimmed_dataset['points_2d'],
#                       trimmed_dataset['camera_intrinsics'][trimmed_dataset['camera_index_of_observations']],
#                       trimmed_dataset['camera_distortions'][trimmed_dataset['camera_index_of_observations']])
# print(jac_from_vmap.shape)

def reprojerr_gen(*args):
    if args[2].shape[0] == MAX_PIXELS:
        return reprojerr(*args)
    else:
        args = list(args)
        return reprojerr_vmap(*args[:-2])

# sparse version
class ReprojNonBatched(nn.Module):
    def __init__(self, camera_extrinsics, points_3d):
        super().__init__()
        self.pose = pp.Parameter(camera_extrinsics)
        self.points_3d = nn.Parameter(points_3d)

    def forward(self, *args):
        return reprojerr_gen(self.pose, self.points_3d, *args)


In [3]:
# densed version
class ReprojBatched(nn.Module):
    def __init__(self, camera_extrinsics, points_3d):
        super().__init__()
        self.pose = nn.Parameter(camera_extrinsics)
        self.points_3d = nn.Parameter(points_3d)

    def forward(self, *args):
        return reprojerr(self.pose, self.points_3d, *args)


In [4]:
from pypose.optim.solver import Krylov
input = [trimmed_dataset['points_2d'],
         trimmed_dataset['camera_intrinsics'][trimmed_dataset['camera_index_of_observations']],
         trimmed_dataset['camera_distortions'][trimmed_dataset['camera_index_of_observations']],
         trimmed_dataset['camera_index_of_observations'],
         trimmed_dataset['point_index_of_observations']]


In [22]:
# %%time
model_non_batched = ReprojNonBatched(trimmed_dataset['camera_extrinsics'].clone(),
                                     trimmed_dataset['points_3d'].clone())

model_non_batched = model_non_batched.to(DEVICE)

strategy_sparse = pp.optim.strategy.Adaptive(damping=1e-6)
optimizer_sparse = pp.optim.LM(model_non_batched, strategy=strategy_sparse, solver=Krylov(rtol=5e-2), reject=3)
for idx in range(10):
    loss = optimizer_sparse.step(input)
    # if (idx + 1) % 10 == 0:
    print('Pose Inversion loss %.7f @ %d it'%(loss, idx))
    if loss < 1e-5:
        print('Early Stopping with loss:', loss.item())
        break


matvec percentage: 49.30%
accept step after 0 retries
solver time percentage: 96.75%
Pose Inversion loss 1023718.3125000 @ 0 it
matvec percentage: 48.67%
accept step after 0 retries
solver time percentage: 92.67%
Pose Inversion loss 545104.0625000 @ 1 it
matvec percentage: 49.43%
reject step 1
matvec percentage: 49.50%
reject step 2
matvec percentage: 49.49%
reject step 3
matvec percentage: 49.48%
accept step after 3 retries
solver time percentage: 99.29%
Pose Inversion loss 678814.2500000 @ 2 it
matvec percentage: 49.39%
accept step after 0 retries
solver time percentage: 95.53%
Pose Inversion loss 664220.4375000 @ 3 it
matvec percentage: 48.35%
accept step after 0 retries
solver time percentage: 91.87%
Pose Inversion loss 504568.0312500 @ 4 it
matvec percentage: 49.11%
accept step after 0 retries
solver time percentage: 95.20%
Pose Inversion loss 478472.4375000 @ 5 it
matvec percentage: 49.44%
reject step 1
matvec percentage: 49.44%
reject step 2
matvec percentage: 49.34%
reject step

In [7]:

model_batched = ReprojBatched(trimmed_dataset['camera_extrinsics'].clone(), trimmed_dataset['points_3d'].clone())

strategy_dense = pp.optim.strategy.Adaptive(damping=1e-6)
optimizer_dense = pp.optim.LM(model_batched, strategy=strategy_dense, sparse=False)
for idx in range(3):
    loss = optimizer_dense.step(input)
    print('Pose Inversion loss %.7f @ %d it'%(loss, idx))
    if loss < 1e-5:
        print('Early Stopping with loss:', loss.item())
        break


TypeError: reprojerr() missing 1 required positional argument: 'point_index'

# Construct SBT from vmap

In [ ]:
i = [[0, 0, 1, 1], [0, 2, 1, 2]]
v = torch.arange(12).view((-1, 1, 3)).to(dtype=torch.float32)
x = pp.sbktensor.sbktensor(i, v, size=(3, 3), dtype=torch.float32)
print(x.to_sparse_coo().shape)
x2 = torch.arange(27).view(9, 3).to(dtype=torch.float32)
x = torch.sparse_coo_tensor(i, v, size=(3, 3), dtype=torch.float32)
print(x.to_sparse_coo().to_dense())


In [ ]:
i = [[0, 0, 1, 1], [0, 2, 1, 2]]
# v = torch.randn(4 * 2 * 2).view((4, 2, 2)).to(dtype=torch.float32)
v = torch.randn(4)
sct = torch.sparse_coo_tensor(i, v)
v2 = torch.randn(4)
sct2 = torch.sparse_coo_tensor(i, v2)
# torch.cat([sct, sct2], dim=1)
# pdt = torch.sparse.mm(v, v2)
pdt = sct @ sct2.T
print(pdt.type())
print(pdt.is_sparse)


In [ ]:
# confirm that the jacobian values from jacrev and vmap match
batch_dim = jac_from_jacrev.shape[0]
index = trimmed_dataset['camera_index_of_observations']
jac_from_jacrev_condensed = jac_from_jacrev[torch.arange(batch_dim), index]
print(jac_from_jacrev_condensed.shape)
if torch.allclose(jac_from_vmap, jac_from_jacrev_condensed):
  print("Jacobian structure sanity check: ok!")
else:
  print("Jacobian structure sanity check: failed!")


In [ ]:
# manually construct SBT from vmap result
import torch
import pypose as pp
from pypose.sparse import sbktensor

def construct_sbt(jac_from_vmap, num_cameras, camera_index):
  # camera_index = torch.from_numpy(camera_index) # for torch.stack
  n = camera_index.shape[0] # num 2D points
  # i = torch.stack([torch.arange(n), camera_index, torch.zeros(n)])
  i = torch.stack([torch.arange(n), camera_index])
  print(i.shape)
  # v = jac_from_vmap[:, None, None, :] # adjust dimension to accomodate for sbt constructor
  v = jac_from_vmap[:, None, :] # adjust dimension to accomodate for sbt constructor
  # return pp.sbktensor(i, v, size=(n, num_cameras, 1), dtype=torch.float32)
  return pp.sbktensor.sbktensor(i, v, size=(n, num_cameras), dtype=torch.float32)

sparse_jac = construct_sbt(jac_from_vmap, len(trimmed_dataset['camera_extrinsics']), trimmed_dataset['camera_index_of_observations'])
dense_jac = sparse_jac.to_dense()
# if torch.allclose(jac_from_jacrev, dense_jac):
#   print("Dense Jacobian <-> Sparse Jacobian: ok!")
# else:
#   print("Dense Jacobian <-> Sparse Jacobian: failed!")


In [ ]:
storage_t = sparse_jac._s
print(f'shape: {storage_t.shape}, dense dim: {storage_t.dense_dim()}, \
sparse: {storage_t.sparse_dim()}')
flattened_coo = pp.hybrid2coo(storage_t) # the flattened tensor
print(flattened_coo.shape)
flattened_coo_T = flattened_coo.T
print(flattened_coo_T.shape)
A = torch.sparse.mm(flattened_coo_T, flattened_coo)
print(A.shape)
print(A.type())
# pdt = torch.matmul(flattened_coo_T, torch.arange(1000).view(-1, 1).to(dtype=torch.float32))
# print(pdt.shape)
# print(pdt.type())


In [ ]:
def sparse_coo_diagonal(t: torch.Tensor):
    indices = t.indices()
    diag_indices = indices[0] == indices[1]
    return t.values()[diag_indices]

def sparse_coo_diagonal_clamp_(t: torch.Tensor, min_value, max_value):
    indices = t.indices()
    diag_indices = indices[0] == indices[1]
    t.values()[diag_indices] = t.values()[diag_indices].clamp_(min_value, max_value)


def sparse_coo_diagonal_add_(t: torch.Tensor, other: torch.Tensor):
    indices = t.indices()
    diag_indices = indices[0] == indices[1]
    t.values()[diag_indices] = t.values()[diag_indices].add_(other)


In [ ]:
diag_indices = (A.indices()[0] == A.indices()[1]).nonzero(as_tuple=True)
# print(diag_indices[0][])
sparse_coo_diagonal_clamp_(A, -100, 100)
print(A.values()[diag_indices[0][:10]])
sparse_coo_diagonal_add_(A, sparse_coo_diagonal(A) * torch.ones(301) * 2)
print(A.values()[diag_indices[0][:10]])


In [ ]:
def jacrev_custom(func, argnums):
  def wrapper(*args, **kwargs):
    jac_vmap = torch.vmap(pp.func.jacrev(func)) # vmap
    gradients = jac_vmap(*args)
    sbt = construct_sbt(gradients, kwargs['num_cols'], kwargs['index'])
    return sbt
  return wrapper


In [ ]:
jac_function_custom = jacrev_custom(reprojerr_vmap, argnums=0)
jac_from_custom = jac_function_custom(trimmed_dataset['camera_extrinsics'][trimmed_dataset['camera_index_of_observations']],
                      trimmed_dataset['points_3d'][trimmed_dataset['point_index_of_observations'], None],
                      trimmed_dataset['points_2d'],
                      trimmed_dataset['camera_intrinsics'][trimmed_dataset['camera_index_of_observations']],
                      trimmed_dataset['camera_distortions'][trimmed_dataset['camera_index_of_observations']],
                      num_cols=len(trimmed_dataset['camera_extrinsics']),
                      index=trimmed_dataset['camera_index_of_observations'])
